[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1AnqmiSw6nWZwJRVqLwaLvP7Uz4yfEDW-?usp=sharing)

In [2]:
!apt-get install libspatialindex-dev
!pip install geopandas rtree

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libspatialindex-c6 libspatialindex6
The following NEW packages will be installed:
  libspatialindex-c6 libspatialindex-dev libspatialindex6
0 upgraded, 3 newly installed, 0 to remove and 38 not upgraded.
Need to get 319 kB of archives.
After this operation, 1,416 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libspatialindex6 amd64 1.9.3-2 [247 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libspatialindex-c6 amd64 1.9.3-2 [55.8 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libspatialindex-dev amd64 1.9.3-2 [16.0 kB]
Fetched 319 kB in 0s (2,304 kB/s)
Selecting previously unselected package libspatialindex6:amd64.
(Reading database ... 126675 files and directories currently installed.)
Preparing to unpack .../libspatialindex6_1.9.3-2_amd64.deb ...
U

In [ ]:
!pip install geopandas fiona shapely pyproj rtree tqdm

In [ ]:
import geopandas as gpd, pandas as pd, numpy as np, sys, os, time, tempfile, pathlib, urllib.request
from fiona import listlayers
from tqdm.auto import tqdm

# Entradas
url_shp  = "https://github.com/ccardenas93/ecociencia/blob/main/Cauce_Principal/Files/Cuencas_n5.shp"
url_gpkg = "https://media.githubusercontent.com/media/ccardenas93/ecociencia/main/Cauce_Principal/Files/rivers_RAISG_corrected2.gpkg"
rivers_gpkg_local = None

# Utils
def to_raw(u): return u.replace("github.com/","raw.githubusercontent.com/").replace("/blob/","/")
def http_download(u,dst):
    u=to_raw(u)
    with urllib.request.urlopen(u) as r, open(dst,"wb") as f:
        while True:
            c=r.read(1024*1024)
            if not c: break
            f.write(c)
    return str(dst)
def is_gpkg(p):
    try:
        with open(p,"rb") as f: return f.read(16).startswith(b"SQLite format 3\x00")
    except: return False
def download_shapefile(base_shp_url,dest_dir):
    exts=[".shp",".shx",".dbf",".prj",".cpg",".qix",".fix"]; stem=pathlib.Path(base_shp_url).name.replace(".shp",""); paths={}
    for ext in exts:
        u=base_shp_url.replace(".shp",ext); p=pathlib.Path(dest_dir)/f"{stem}{ext}"
        try: http_download(u,p); paths[ext]=p
        except: paths[ext]=None
    if not all(paths[k] for k in [".shp",".dbf",".shx"]):
        missing=[k for k,v in paths.items() if v is None]; sys.exit(f"Shapefile incompleto desde GitHub. Faltan: {', '.join(missing)}")
    return str(paths[".shp"])
def get_gpkg(dest_dir):
    if rivers_gpkg_local and os.path.exists(rivers_gpkg_local):
        if not is_gpkg(rivers_gpkg_local): sys.exit(f"GPKG local no válido: {rivers_gpkg_local}")
        return rivers_gpkg_local
    p=pathlib.Path(dest_dir)/pathlib.Path(url_gpkg).name
    try: http_download(url_gpkg,p)
    except: sys.exit("No pude descargar el GPKG desde GitHub.")
    if not is_gpkg(p): sys.exit("El archivo descargado no es un GPKG válido (¿puntero LFS?).")
    return str(p)

# Descargas / rutas
tmp=tempfile.mkdtemp(prefix="ecociencia_")
basins_path=download_shapefile(url_shp,tmp)
rivers_path=get_gpkg(tmp)

# Parámetros
basin_layer=listlayers(basins_path)[0]
river_layer=listlayers(rivers_path)[0]
out_gpkg,out_layer,out_csv="principal_river_by_basin1.gpkg","principal1","principal1_river_lengths.csv"
target_epsg=4326
simplify_tolerance_m=0
predicate="intersects"
VERBOSE=False; LOG_EVERY_N=50

# Carga y reproyección
t0=time.perf_counter(); tqdm.write("Cargando datos…")
b=gpd.read_file(basins_path,layer=basin_layer,encoding="latin1")
r=gpd.read_file(rivers_path,layer=river_layer)
for g,n in [(b,"basins"),(r,"rivers")]:
    if g.crs is None: sys.exit(f"{n} sin CRS")
tqdm.write("Reproyectando a EPSG:4326…")
b=b.to_crs(target_epsg); r=r.to_crs(target_epsg)

if "BASIN_ID" not in b: b=b.reset_index(drop=False).rename(columns={"index":"BASIN_ID"})
b=b[["BASIN_ID","geometry"]]
need=["MAIN_RIV","ORD_CLAS","geometry"]; missing=[c for c in need if c not in r.columns]
if missing: sys.exit(f"Faltan columnas en ríos: {missing}")
if "DIS_AV_CMS" not in r: r["DIS_AV_CMS"]=pd.NA
r=r[["MAIN_RIV","ORD_CLAS","DIS_AV_CMS","geometry"]]
if simplify_tolerance_m>0:
    tqdm.write(f"Simplificando cuencas (tol={simplify_tolerance_m} m)…")
    b["geometry"]=b.geometry.simplify(simplify_tolerance_m,preserve_topology=True)

# sjoin progresivo
tqdm.write("Emparejando ríos–cuencas con progreso…")
si=r.sindex; parts=[]
for bid,geom in tqdm(zip(b["BASIN_ID"],b.geometry), total=len(b), desc="sjoin progreso"):
    idx=list(si.query(geom))
    if not idx: continue
    cand=r.iloc[idx].copy()
    mask=getattr(cand.geometry,predicate)(geom)
    cand=cand.loc[mask]
    if cand.empty: continue
    cand["BASIN_ID"]=bid
    parts.append(cand[["BASIN_ID","MAIN_RIV","ORD_CLAS","DIS_AV_CMS","geometry"]])
if not parts: sys.exit("No hay ríos que intersecten cuencas.")
tagged=gpd.GeoDataFrame(pd.concat(parts,ignore_index=True), geometry="geometry", crs=r.crs)
tqdm.write(f"sjoin (progresivo) listo: {len(tagged)} emparejamientos.")

# mínimo ORD_CLAS y candidatos
tqdm.write("Calculando ORD_CLAS mínimo por cuenca…")
min_ord=tagged.groupby("BASIN_ID")["ORD_CLAS"].min().rename("min_ord_in_basin")
tagged=tagged.merge(min_ord,on="BASIN_ID")
cands=tagged[tagged["ORD_CLAS"]==tagged["min_ord_in_basin"]].copy()
tqdm.write(f"Candidatos (mín ORD por cuenca): {len(cands)}")

# loop por cuenca (auto: mayor caudal, luego mayor longitud)
cands["len_km_total"]=cands.length/1000
rows=[]; geoms=[]; basin_geom=dict(zip(b["BASIN_ID"],b.geometry))
time_clip=time_group=time_choose=0.0
cuencas=sorted(cands["BASIN_ID"].unique().tolist()); tqdm.write(f"Procesando {len(cuencas)} cuencas…")

for idx,bid in enumerate(tqdm(cuencas, desc="Cuencas")):
    grp=cands[cands["BASIN_ID"]==bid]; g_basin=basin_geom[bid]
    tA=time.perf_counter(); clipped=gpd.clip(grp.set_geometry("geometry"), g_basin); time_clip+=time.perf_counter()-tA
    if clipped.empty:
        tB=time.perf_counter()
        agg=grp.groupby("MAIN_RIV",as_index=False).agg(len_km_in_basin=("len_km_total","sum"), dis_av_cms=("DIS_AV_CMS","mean"), ord_clas=("ORD_CLAS","min"))
        time_group+=time.perf_counter()-tB
    else:
        clipped["len_km_in_basin"]=clipped.length/1000
        tB=time.perf_counter()
        agg=clipped.groupby("MAIN_RIV",as_index=False).agg(len_km_in_basin=("len_km_in_basin","sum"), dis_av_cms=("DIS_AV_CMS","mean"), ord_clas=("ORD_CLAS","min"))
        time_group+=time.perf_counter()-tB
    tC=time.perf_counter()
    agg["_Q"]=agg["dis_av_cms"].fillna(-1)
    agg=agg.sort_values(["_Q","len_km_in_basin"], ascending=[False,False]).drop(columns="_Q").reset_index(drop=True)
    mr,L,Q,O=agg.loc[0,["MAIN_RIV","len_km_in_basin","dis_av_cms","ord_clas"]]
    mode="auto_max_Q"
    time_choose+=time.perf_counter()-tC
    rows.append({"BASIN_ID":bid,"MAIN_RIV":mr,"len_km_in_basin":L,"dis_av_cms":Q,"ord_clas_elegido":int(O),"modo":mode})
    if not clipped.empty:
        gsel=clipped[clipped["MAIN_RIV"]==mr]; ggeom=gsel.dissolve(by="MAIN_RIV").geometry.iloc[0]
        geoms.append((bid,mr,L,Q,int(O),ggeom))
    if (idx+1)%50==0: tqdm.write(f"[{idx+1}/{len(cuencas)}] t_clip={time_clip:,.1f}s t_group={time_group:,.1f}s t_choose={time_choose:,.1f}s")

principal_tbl=pd.DataFrame(rows)
pg=gpd.GeoDataFrame(
    [{"BASIN_ID":bid,"MAIN_RIV":mr,"len_km_in_basin":L,"dis_av_cms":Q,"ord_clas_min_basin":O,"geometry":geom}
     for (bid,mr,L,Q,O,geom) in geoms],
    geometry="geometry", crs=b.crs
) if geoms else gpd.GeoDataFrame(columns=["BASIN_ID","MAIN_RIV","len_km_in_basin","dis_av_cms","ord_clas_min_basin","geometry"], geometry="geometry", crs=b.crs)

try:
    if os.path.exists(out_gpkg): os.remove(out_gpkg)
except: pass
tqdm.write("Escribiendo GPKG y CSV…")
pg.to_file(out_gpkg, layer=out_layer, driver="GPKG")
principal_tbl.to_csv(out_csv, index=False)
tqdm.write("¡Listo!")

Cargando datos…
Reproyectando a EPSG:4326…
Emparejando ríos–cuencas con progreso…


sjoin progreso:   0%|          | 0/86 [00:00<?, ?it/s]